# Contoh Asas AutoGen

Dalam contoh kod ini, anda akan menggunakan Rangka Kerja AI [AutoGen](https://aka.ms/ai-agents/autogen) untuk mencipta ejen asas.

Matlamat contoh ini adalah untuk menunjukkan kepada anda langkah-langkah yang akan kita gunakan kemudian dalam contoh kod tambahan apabila melaksanakan corak ejen yang berbeza.


## Importkan Pakej Python yang Diperlukan


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Membuat Klien

Dalam contoh ini, kita akan menggunakan [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) untuk akses kepada LLM.

`model` ditetapkan sebagai `gpt-4o-mini`. Cuba tukar model kepada model lain yang tersedia di pasaran GitHub Models untuk melihat hasil yang berbeza.

Sebagai ujian ringkas, kita hanya akan menjalankan arahan mudah - `What is the capital of France`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Mendefinisikan Ejen

Sekarang kita telah menyediakan `client` dan mengesahkan bahawa ia berfungsi, mari kita cipta `AssistantAgent`. Setiap ejen boleh diberikan:  
**name** - Nama pendek yang berguna untuk merujuknya dalam aliran berbilang ejen.  
**model_client** - Klien yang anda cipta dalam langkah sebelumnya.  
**tools** - Alat yang tersedia yang boleh digunakan oleh Ejen untuk menyelesaikan tugasan.  
**system_message** - Metaprompt yang mentakrifkan tugasan, tingkah laku, dan nada LLM.  

Anda boleh menukar mesej sistem untuk melihat bagaimana LLM bertindak balas. Kita akan membincangkan `tools` dalam Pelajaran #4.  


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Jalankan Ejen

Fungsi di bawah akan menjalankan ejen. Kami menggunakan kaedah `on_message` untuk mengemas kini keadaan Ejen dengan mesej baharu.

Dalam kes ini, kami mengemas kini keadaan dengan mesej baharu daripada pengguna iaitu `"Rancang percutian cerah yang hebat untuk saya"`.

Anda boleh menukar kandungan mesej untuk melihat bagaimana LLM memberikan respons yang berbeza.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat yang kritikal, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
